<a href="https://colab.research.google.com/github/ecker-lab/TreeLearn/blob/main/TreeLearn_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Example Notebook for TreeLearn


In [ ]:
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/drive/1xbsup3VbgpN2eLbz7vVnfmoQ5yCRaBHm?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

Thank you for your interest in our TreeLearn method! With this notebook and google colab, you can try out the pipeline for segmenting a forest point cloud without installing anything on your own computer!

You need to be signed in with your google account. Please also make sure that you are connected to a gpu runtime by by selecting 'runtime' change runtime to e.g. T4 GPU. The following code snippet will show a table with gpu information if you are connnected to a gpu runtime. To run the code snippet, simply click on the left edge. or press (Ctrl + enter) after selecting it.

In [ ]:
!nvidia-smi

Tue Nov  7 12:25:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

The following two code snippets are necessary to set up the environment and download the model checkpoints. Simply run them before continuing. It takes 2 to 3 minutes.

In [ ]:
# %%capture
# install environment
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install timm==0.6.12
!pip install tensorboard
!pip install gdown
!pip install tensorboardX

!pip install munch==2.5.0
!pip install pandas==2.0.0
!pip install plyfile==0.9
!pip install pyyaml==6.0
!pip install scikit-learn==1.2.2
!pip install six==1.16.0
!pip install tqdm==4.65.0
!pip install open3d-cpu==0.17.0 --default-timeout=100
!pip install jakteristics==0.5.1
!pip install shapely==2.0.1
!pip install geopandas==0.12.2
!pip install alphashape==1.3.1
!pip install plotly-express==0.4.1 --default-timeout=100
!pip install spconv-cu114 --default-timeout=100

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 36.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.

In [ ]:
%%capture
!git clone https://github.com/ecker-lab/TreeLearn.git
%cd TreeLearn
!pip install -e .
%cd ..
!mkdir data
!mkdir checkpoints
!mkdir pipeline
!mkdir pipeline/forests
#!python TreeLearn/tree_learn/util/download.py --dataset_name checkpoints --root_folder /content/checkpoints

import gdown
# link = "https://drive.google.com/uc?id=1yJ8LOyhvt0OdVMK1WHeVDFfe2N0RGC9J"
# folder = /content/drive/MyDrive/share/TreeLearn_colab_example/finetuned_checkpoint_classifier.pth
# gdown.download(link, folder)
# link = "https://drive.google.com/uc?id=1yJ8LOyhvt0OdVMK1WHeVDFfe2N0RGC9J"
# folder = /content/drive/MyDrive/share/TreeLearn_colab_example/finetuned_checkpoint_pointwise_prediction.pth
# gdown.download(link, folder)
!cp /content/drive/MyDrive/share/TreeLearn_colab_example/finetuned_checkpoint_classifier.pth /content/checkpoints/finetuned_checkpoint_classifier.pth
!cp /content/drive/MyDrive/share/TreeLearn_colab_example/finetuned_checkpoint_pointwise_prediction.pth /content/checkpoints/finetuned_checkpoint_pointwise_prediction.pth
%cd TreeLearn

## Pipeline

We first need to decide which point cloud we want to segment. The following code snippet downloads an example point cloud segment that we did not train on. The size is 40 x 40 meters.

If you want to try out another forest point cloud, replace the download with your own. Make sure that the file is in the .npy or the .txt file format and the total size of the forest stretch should be around 1600 m^2 at maximum. The point cloud should only contain the three columns with x, y and z values and no labels. Please note that with a forest point cloud of this size the segmentation took in our runs around 15 minutes in google colab due to limited computation resources.

Please note that our models have been trained on tls/mls data of forests dominated by beech. **We expect that for a good performance on e.g. uav data and other forest types, finetuning the models is necessary.**

In [ ]:
import gdown
forest_name = "plot_7_cut.npy"
link = "https://drive.google.com/uc?id=1V6-JvDnQn1_koAdbcquSgP9B81xWrNWs"
folder = "/content/pipeline/forests/" + forest_name
gdown.download(link, folder)

Downloading...
From: https://drive.google.com/uc?id=1V6-JvDnQn1_koAdbcquSgP9B81xWrNWs
To: /content/pipeline/forests/plot_7_cut.npy
100%|██████████| 92.3M/92.3M [00:00<00:00, 167MB/s]


'/content/pipeline/forests/plot_7_cut.npy'


To run the TreeLearn pipeline interactively in google colab, we import the function run_treelearn_pipeline. This function takes as argument the config dict. We import the pipeline.yaml as the config dict and print it.

We adjust some entries in the config dict to fit to the setting in google colab and speed up the pipeline. We also initialize the logger so that the progress in the pipeline is printed.


In [ ]:
import sys
sys.path.append("/content/TreeLearn/tools/pipeline")
from pipeline import run_treelearn_pipeline
import argparse, pprint
from tree_learn.util import get_config

config_path = "/content/TreeLearn/configs/pipeline/pipeline.yaml"
config = get_config(config_path)

# adjust config
config.forest_path = "/content/pipeline/forests/" + forest_name
config.dataset_test.data_root = "/content/pipeline/tiles"
config.tile_generation = True
config.pretrain_classifier = "/content/checkpoints/finetuned_checkpoint_classifier.pth"
config.pretrain_pointwise = "/content/checkpoints/finetuned_checkpoint_pointwise_prediction.pth"
config.sample_generation.stride = 0.9 # small overlap
config.shape_cfg.outer_remove = False # default value = 11
config.save_cfg.save_treewise = False
config.save_cfg.return_type = "voxelized_and_denoised"
print(pprint.pformat(config.toDict(), indent=2))

import logging
logger = logging.getLogger("softgroup")
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
logging.basicConfig()
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel(logging.INFO)

{ 'dataloader': {'batch_size': 1, 'num_workers': 1},
  'dataset_test': { 'data_root': '../datasets_simlink/data_trees/validation_data/tiles',
                    'inner_square_edge_length': 8,
                    'training': False,
                    'use_tree_height_in_offset': True},
  'forest_path': '../datasets_simlink/data_trees/test/forests/L1W.npy',
  'fp16': True,
  'global_filtering': True,
  'grouping': {'npoint_thr': 100, 'radius': 0.6, 'tree_conf_thresh': 0.5},
  'local_filtering': False,
  'model': { 'channels': 32,
             'dim_coord': 3,
             'dim_feat': 4,
             'fixed_modules': [],
             'kernel_size': 3,
             'max_num_points_per_voxel': 3,
             'mode': 'pointwise',
             'n_voxels_in_each_direction': 20,
             'num_blocks': 7,
             'spatial_shape': [500, 500, 1000],
             'use_coords': False,
             'use_feats': False,
             'voxel_size': 0.1},
  'model_classifier': { 'dim_feat': 32,

INFO:softgroup:Test log


[<StreamHandler stdout (INFO)>]


After having set all the correct settings in the config file, it remains to run the pipeline. Please keep in mind that fully running it for the example point cloud takes around 15 minutes.



In [ ]:
# run pipeline
run_treelearn_pipeline(config)
# tile generation runtime ~ 8-9 min

2023-11-07 12:05:49,724 - INFO - Munch({'sample_generation': Munch({'voxel_size': 0.1, 'search_radius_features': None, 'inner_edge': 8, 'outer_edge': 11, 'stride': 0.9, 'sample_generator': Munch({'n_neigh_sor': 2, 'multiplier_sor': 1, 'rad': 0.2086, 'npoints_rad': 5})}), 'model': Munch({'mode': 'pointwise', 'kernel_size': 3, 'channels': 32, 'num_blocks': 7, 'use_feats': False, 'use_coords': False, 'dim_coord': 3, 'dim_feat': 4, 'max_num_points_per_voxel': 3, 'fixed_modules': [], 'spatial_shape': [500, 500, 1000], 'n_voxels_in_each_direction': 20, 'voxel_size': 0.1}), 'grouping': Munch({'npoint_thr': 100, 'radius': 0.6, 'tree_conf_thresh': 0.5}), 'forest_path': '/content/pipeline/forests/plot_7_cut.npy', 'pretrain_pointwise': '/content/checkpoints/finetuned_checkpoint_pointwise_prediction.pth', 'pretrain_classifier': '/content/checkpoints/finetuned_checkpoint_classifier.pth', 'fp16': True, 'tile_generation': True, 'global_filtering': True, 'local_filtering': False, 'model_classifier': M

INFO:softgroup:Munch({'sample_generation': Munch({'voxel_size': 0.1, 'search_radius_features': None, 'inner_edge': 8, 'outer_edge': 11, 'stride': 0.9, 'sample_generator': Munch({'n_neigh_sor': 2, 'multiplier_sor': 1, 'rad': 0.2086, 'npoints_rad': 5})}), 'model': Munch({'mode': 'pointwise', 'kernel_size': 3, 'channels': 32, 'num_blocks': 7, 'use_feats': False, 'use_coords': False, 'dim_coord': 3, 'dim_feat': 4, 'max_num_points_per_voxel': 3, 'fixed_modules': [], 'spatial_shape': [500, 500, 1000], 'n_voxels_in_each_direction': 20, 'voxel_size': 0.1}), 'grouping': Munch({'npoint_thr': 100, 'radius': 0.6, 'tree_conf_thresh': 0.5}), 'forest_path': '/content/pipeline/forests/plot_7_cut.npy', 'pretrain_pointwise': '/content/checkpoints/finetuned_checkpoint_pointwise_prediction.pth', 'pretrain_classifier': '/content/checkpoints/finetuned_checkpoint_classifier.pth', 'fp16': True, 'tile_generation': True, 'global_filtering': True, 'local_filtering': False, 'model_classifier': Munch({'mode': 'cla

2023-11-07 12:05:49,726 - INFO - #################### generating tiles ####################


INFO:softgroup:#################### generating tiles ####################


2023-11-07 12:05:49,729 - INFO - voxelizing data and features...


...

INFO:softgroup:plot_7_cut: #################### getting predicted instances ####################
9129it [00:00, 87778.17it/s]
100%|██████████| 1318964/1318964 [00:06<00:00, 189286.57it/s]


clustering
2023-11-07 12:17:17,401 - INFO - plot_7_cut: #################### Run classifier on preliminary instances ####################


INFO:softgroup:plot_7_cut: #################### Run classifier on preliminary instances ####################
100%|██████████| 51/51 [00:02<00:00, 18.72it/s]


2023-11-07 12:17:32,384 - INFO - plot_7_cut: #################### Saving ####################


INFO:softgroup:plot_7_cut: #################### Saving ####################


If everything has run as expected, the segmented point cloud is now saved in the .ply format with labels in the directory /content/pipeline/results. It is also saved in the .npy format. You can easily download it by right-clicking and selecting download.

In [ ]:
import numpy as np
coords = np.load("/content/pipeline/results/" + forest_name.split(".")[0] + "/full_forest/" + forest_name)
print("Number of identified trees:", len(np.unique(coords[:,3])))


Number of identified trees: 33
